In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [ ]:
#Load dataset
churn = pd.read_excel("Churn.xlsx")
churn = churn[churn["TotalCharges"] != " "]
churn["TotalCharges"] = churn["TotalCharges"].astype(float)

> Impact of tenure on Churn:

In [ ]:
tenure = churn[["tenure","Churn"]].copy()
tenure.Churn[tenure.Churn == 1] = "Yes"
tenure.Churn[tenure.Churn == 0] = "No"

In [ ]:
tenure1 = tenure[tenure.tenure <= 6]
tenure2 = tenure[np.logical_and(tenure.tenure > 6,tenure.tenure <= 20)]
tenure3 = tenure[np.logical_and(tenure.tenure > 20,tenure.tenure <= 40)]
tenure4 = tenure[np.logical_and(tenure.tenure > 40,tenure.tenure <= 60)]
tenure5 = tenure[tenure.tenure > 60]

In [ ]:
plt.hist(tenure1.Churn, histtype="step")
plt.hist(tenure2.Churn, histtype="step")
plt.hist(tenure3.Churn, histtype="step")
plt.hist(tenure4.Churn, histtype="step")
plt.hist(tenure5.Churn, histtype="step")

We see that customers who have been customers for shorter periods are more likely to leave.
This histogram not readable, make better.

> Machine learning:

In [ ]:
train = churn.iloc[0:6000].drop("customerID", axis=1)
test =  churn.iloc[6001:7000].drop("customerID", axis=1)

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(train.drop("Churn", axis=1), train.Churn)

predicted_churn = classifier.predict(test.drop("Churn", axis=1))

In [ ]:
matched = predicted_churn == test.Churn

In [ ]:
plt.hist(matched)

Using all the features available, we see we can predict 80% of churn with this model of random forest

> See what features matter the most